In [ ]:
# Standard library imports
import os
import random
import warnings

# Third-party imports
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import backend as K
from PIL import Image
from skimage.io import imread, imshow
from skimage.transform import resize
import nbimporter2
from unet_model import UNET
from sklearn.metrics import classification_report


# Setting up matplotlib to work interactively in a Jupyter environment
%matplotlib inline

# Setting the seed for reproducibility
seed_value = 42
random.seed(seed_value)  # For Python's built-in random module
np.random.seed(seed_value)  # For NumPy
tf.random.set_seed(seed_value)  # For TensorFlow

# Don't show warning messages
warnings.filterwarnings('ignore')


In [1]:
###Performance Metrics
def Test_(model, test_list, test_gen):
   
    label_sets = []
    iou = []
    f1=[]
    acc=[]
    recl =[]
    prec=[]
    for i, batch in enumerate(test_gen):
        labels = batch[1]
        test_img = batch[0]
        

      
        test_preds = model.predict(test_img, verbose=0)

        preds_test_thresh = (test_preds >= 0.5).astype(np.uint8)
        
        jcard_sim = jaccard_similarity( labels,preds_test_thresh)
        iou.append(jcard_sim)
        f1.append(f1_score( labels,preds_test_thresh))
        acc.append(accuracy( labels,preds_test_thresh))
        recl.append(recall( labels,preds_test_thresh))
        prec.append(precision( labels,preds_test_thresh))
        

        if len(iou) >= len(test_list):
            break
    average_similarity = np.mean(iou)
    average_acc = np.mean(acc)
    average_f1 = np.mean(f1)
    average_recall = np.mean(recl)
    average_precl = np.mean(prec)
     # Calculate the standard deviation for each metric
    std_similarity = np.std(iou)
    std_acc = np.std(acc)
    std_f1 = np.std(f1)
    std_recall = np.std(recl)
    std_prec = np.std(prec)

    return average_similarity, average_acc, average_f1, average_recall, average_precl, std_similarity, std_acc, std_f1, std_recall, std_prec

def f1_score(y_true, y_pred):
    """Calculate the F1 score given the true and predicted labels."""
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    if tp + fp ==0  or tp + fn == 0:
        return 0
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if precision + recall==0:
        return 0
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

def accuracy(y_true, y_pred):
    """Calculate the accuracy given the true and predicted labels."""
    num_correct = np.sum(y_true == y_pred)
    num_total = int(y_true.shape[0]*y_true.shape[1]*y_true.shape[2])
    acc = num_correct / num_total
    return acc

def recall(y_true, y_pred):
    """Calculate the recall given the true and predicted labels."""
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    if tp + fn == 0:
        return 0
    recall = tp / (tp + fn)
#     report = classification_report(y_true.flatten(), y_pred.flatten(), output_dict=True)
#     print(report)
#     recall = report['True']['recall']
    return recall

def precision(y_true, y_pred):
    """Calculate the recall given the true and predicted labels."""
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    if tp + fp == 0:
        return 0
    recall = tp / (tp + fp)
    return recall

def jaccard_similarity(y_true, y_pred):
    """Calculate the Jaccard similarity given the true and predicted labels."""
    intersection = np.sum((y_true == 1) & (y_pred == 1))
    union = np.sum((y_true == 1) | (y_pred == 1))
    jaccard = (intersection+1) / (union+1)
    return jaccard

In [3]:
def show_metrics(base_ptch_1024, test_list_large, test_gen_large):
    sim, acc_, f1_, rec, preci, std_sim, std_acc, std_f1, std_rec, std_preci = Test_(base_ptch_1024.model, test_list_large, test_gen_large)
    print(f"Jaccard sim: {round(sim, 3)} ± {round(std_sim, 3)}, Accuracy: {round(acc_, 3)} ± {round(std_acc, 3)}, Precision: {round(preci, 3)} ± {round(std_preci, 3)}, F1-score: {round(f1_, 3)} ± {round(std_f1, 3)}, Recall: {round(rec, 3)} ± {round(std_rec, 3)}")

In [4]:
def pred_image(IMG_HEIGHT, model, fn):
    
    image = cv2.imread(data_path +fn, 0)
    image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))/255.
    image_ = (image - mean)/std
    
    label_img  = cv2.imread(lab_path +fn, 0)
    label_img = cv2.resize(label_img, (IMG_WIDTH, IMG_HEIGHT))/255.
    
    
    test_preds = model.predict(image_.reshape(1,IMG_WIDTH,IMG_HEIGHT,1), verbose = 0)
    preds_test_thresh = (test_preds >= 0.5).astype(np.uint8)
    
    fig, axes = plt.subplots(ncols=3,figsize=(10, 10))

    # Display the images on the axes
    axes[0].imshow(image, cmap='gray')
    axes[1].imshow(label_img, cmap='gray')
    axes[2].imshow(preds_test_thresh.reshape(IMG_WIDTH,IMG_HEIGHT), cmap='gray')

    # Set the titles for each image
    axes[0].set_title('Full sized Image ')
    axes[1].set_title('Label')
    axes[2].set_title('Prediciton')
    


NameError: name 'os' is not defined

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:

      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    

## Sample Usage

In [ ]:
# Create empty arrays
IMG_HEIGHT = 1024
IMG_WIDTH = 1024
IMG_CHANNELS = 1
per_image_norm = False

# Add these constants for brightness, saturation, and hue adjustments
BRIGHTNESS = 0.2
SATURATION = 0.3
HUE = 0.25

# Define Paths
data_path = r'< full_train_data_path >' 
lab_path =  r'< full_test_data_path >'
part_data_path = r'< part_train_data >'   
part_lab_path = r'< part_test_data >'

part_train_list = os.listdir(part_data_path)
test_list = os.listdir(data_path)

train_gen, val_gen, test_gen_large = preprocess_and_load(data_path, part_data_path, part_lab_path, lab_path, train_list, test_list)

test_data_path_large = r'<PATH_TO_YOUR_TEST_IMAGES>' 
test_lab_path_large =  r'<PATH_TO_YOUR_TEST_LABELS'

# Load the model
base_ptch_1024 = UNET(IMG_HEIGHT)
base_ptch_1024.model.load_weights('path_to_your_trained_model.h5')

test_data_list = os.listdir(test_data_path)
ind_ = random.randint(0, len(test_data_list)-1)

show_metrics(base_ptch_1024, test_list_large, test_gen_large)
pred_image(base_ptch_1024.model,test_data_list[ind_] )